In [1]:
import wandb
run = wandb.init()
artifact = run.use_artifact('ain-space/gaia/model-tzmvobqh:v0', type='model')
artifact_dir = artifact.download()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bernddoser. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


In [7]:
import torch
from spherinator.models import (
    ConvolutionalDecoder1D,
    ConvolutionalEncoder1D,
    AutoencoderPure,
)

encoder = ConvolutionalEncoder1D(input_dim=344, output_dim=3)
decoder = ConvolutionalDecoder1D(input_dim=3, output_dim=344)
model = AutoencoderPure(encoder=encoder, decoder=decoder)
model

ckpt = "/home/doserbd/git/Spherinator-main/gaia/artifacts/model-tzmvobqh:v0/model.ckpt"
checkpoint = torch.load(ckpt, weights_only=True, map_location="cpu")
# checkpoint["state_dict"]

model.load_state_dict(checkpoint["state_dict"])
model.eval()
# onnx = torch.onnx.dynamo_export(model, model.example_input_array)
# onnx.save("model.onnx")

# encoder.load_state_dict(checkpoint["state_dict"]["encoder"])
# encoder.eval()

export_options = torch.onnx.ExportOptions(dynamic_shapes=True)
onnx = torch.onnx.dynamo_export(model.encoder, model.example_input_array, export_options=export_options)
onnx.save("encoder.onnx")
onnx = torch.onnx.dynamo_export(model.decoder, torch.randn(2, 3, device="cpu"), export_options=export_options)
onnx.save("decoder.onnx")

/home/doserbd/git/Spherinator-main/.venv/lib/python3.9/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
/home/doserbd/git/Spherinator-main/.venv/lib/python3.9/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'decoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['decoder'])`.
/home/doserbd/git/Spherinator-main/.venv/lib/python3.9/site-packages/torch/onnx/_internal/_exporter_legacy.py:116: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
/home/doserbd/git/Spherinator-main/.venv/lib/python3.9/site-packages/torch/onnx/_internal/fx/passes/readabil